In [1]:
from PIL import Image
import os
from pathlib import Path
from utils import *
from single_multiple_efficientNet import *
from engine import *
from data_transforms import create_transform
from dataloaders import *

change_to_disk()
data_dir = Path("Quality/data/new_data/split_1k")
models_path = Path('Quality/Models_double/')
check_dir(models_path)

In [2]:
image_shape = (1080, 1920)
resize_factor = 0.3
resize = np.multiply(image_shape,resize_factor)
resize = [int(resize[0]), int(resize[1])]
BATCH_SIZE = 16

data_transform = create_transform(resize=resize, rotate=10, flip_h = True, color_dev=True, transf_tensor=True, normalize=True, sp_noise=True, gauss_noise=True)

train_dl, validation_dl, test_dl, train_data, validation_data, test_data = create_dataloaders_multilabel(data_dir, transform = data_transform, batch_size=BATCH_SIZE, num_workers= NUM_WORKERS)

In [3]:
NUM_EPOCHS = 100
lr = 0.001
device = get_device()

effnetb0 = EfficientNetB0()

loss_fn_binary = nn.BCELoss()
loss_fn_multiclass = nn.CrossEntropyLoss()

In [7]:
# Get a single image batch
image_batch, label_batch = next(iter(train_dl))
image_batch.shape, label_batch.shape

(torch.Size([16, 3, 324, 576]), torch.Size([16, 3]))

In [27]:
label_batch[:,0].unsqueeze(dim=1)

tensor([[0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.]])

In [ ]:
model_name = "EfficientNetB0"
model_folder_path = models_path / model_name
check_dir(model_folder_path)
model_name_folder_path = model_folder_path / model_name

#Train EfficientNet B0
effnetb0.to(device)
nadam_optim = torch.optim.NAdam(params=effnetb0.parameters(), lr=lr)
train_effnetb0_results, train_time_effnetb0 = train(effnetb0, train_dl, validation_dl, optimizer=nadam_optim, loss_fn_binary=loss_fn_binary, loss_fn_multiclass=loss_fn_multiclass, epochs=NUM_EPOCHS, name_save=model_name_folder_path, device=device)